In [1]:
import json
from adabas.api import *
from adabas.datamap import *

In [2]:
from datetime import date
mask_date = lambda data, mask='%d-%m-%Y': date.fromordinal(int(data[2:])-364).strftime(mask)

In [3]:
#campo elementar de um grupo periódico ou campo elementar múltiplo
def get_periodic(isn = 0   # isn a ser pesquisado
                ,gsn = ''  # grupo short name ou elementar multiplo    - FDT
                ,esn = ''  # elementar short name - FDT
                ,lgn = ''  # elementar long-name  - DDM
                ,lsn = 0   # length elementar     - DDM
                ,upk = ''  # unpack para colunas packeadas '' or True
                ,scl = ''  # scale or ''          - DDM
                ,Int = ''  # True para numéricos
                 ):
    ret = ''
    try:
        upk = ',U' if upk else ''
        Str = '"'
        a.cb.isn=isn                 
        if scl or Int:
            Int = 'int('
            Str = ''
            if scl:
                scl = ') * .{0:>0{1}}'.format('1', scl)
            else:
                scl = ')'

        a.fb.value='{}C,002,B.'.format(gsn)
        a.get(isn=a.cb.isn)
        if a.rb.value:
            occ=int(a.rb.value.encode('hex') ) # count occurs group
            exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
            row.buffer = a.rb
            a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
            a.get(isn=a.cb.isn)
            ret = eval("""'"{}": [{}]'.format({})""".format( lgn, (Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))    
          #  ret = ret[:-3] + ']'
            ret = ret.replace(', ]', ']')
    except DatabaseError, (error,apa):
        return error.split(':')[1]
    return ret

In [4]:
#campo multiplo de um grupo periódico
def get_per_mult(isn = 0   # isn a ser pesquisado
                ,gsn = ''  # grupo short name     - FDT
                ,esn = ''  # elementar short name - FDT
                ,lgn = ''  # elementar long-name  - DDM
                ,lsn = 0   # length elementar     - DDM
                ,upk = ''  # unpack para colunas packeadas '' or True
                ,scl = ''  # scale or ''          - DDM
                ,Int = ''  # True para numéricos
                 ):
    ret = ''
    try:
        upk = ',U' if upk else ''
        Str = '"'
        a.cb.isn=isn
        if scl or Int:
            Int = 'int('
            Str = ''
            if scl:
                scl = ') * .{0:>0{1}}'.format('1', scl)
            else:
                scl = ')'
        lst = []        
        a.fb.value='{}C,002,B.'.format(gsn)
        a.get(isn=a.cb.isn)
        if a.rb.value:
            gocc=int(a.rb.value.encode('hex') ) # count occurs group
            for o in range(1,gocc+1):
                a.fb.value='{}{}C,002,B.'.format(esn,o)
                a.get(isn=a.cb.isn)
                if a.rb.value:
                    occ=int(a.rb.value.encode('hex') ) # count occurs elementar
                    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
                    row.buffer = a.rb
                    a.fb.value='{}{}(1-{}),{}{}.'.format(esn, o, occ, lsn, upk)
                    a.get(isn=a.cb.isn)
                    ret = eval("""'[{}]'.format({})""".format((Str+'{}'+Str+', ') * occ, ('{i}row.{ln}{}{sc}, '* occ).format(i=Int,ln=lgn,sc=scl,*range(1,occ+1))))
                    lst.append(eval(ret))
                else:
                    break
    except DatabaseError, (error,apa):
        return error.split(':')[1]
    return '"{}": {}'.format(lgn, lst)            

In [45]:
DBID=12;FNR=11 
STARTISN=282
RCOUNT=5

row=Datamap('rows',
    String('personnel_id',  8), # aa 1
    String('first_name',   20), # ac 2
    String('middle_name',  20), # ad 3
    String('name',         20), # ae 4
    String('birth',         8), # ah 5
    String('country',       3), # al 6
    String('area_code',     6), # an 7
    String('phone',        15), # am 8
    String('dept',          6), # ao 9
    String('job_title',    25)) # ap 10

In [46]:
lines=''                                         
extraline=''                                     
alter=False                                      
c1=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)   
c1.cb.dbid=DBID
c1.cb.fnr=FNR
c1.cb.cid='1010'                                 
c1.fb.value='AA,AC,AD,AE,AH,8,U,AL,AN,AM,AO,AP.' 

In [47]:
a=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)
a.cb.dbid=12              
a.cb.fnr=11                            

In [48]:
c1.cb.isn=STARTISN                                                                 
# use emp Datamap on record buffer                                                 
row.buffer=c1.rb                                                                   
row.offset=0                                                                       
                                                                                  
count=0                                                                            
                                                                                   
try:                                                                               
    for count in range(RCOUNT):                                                    
        c1.readByIsn(getnext=1)                                                    
                                                                                    
        lines += """{}"{}": {}, "{}": "{}", "{}": "{}", "{}": "{}","""\
                 """"{}": "{}", "{}": "{}", "{}": "{}","""\
                 """"{}": "{}", "{}": "{}", "{}": "{}","""\
                 """"{}": "{}", {}, {}{}\n""".format("{"
                                            ,'isn'         , c1.cb.isn           
                                            ,'personnel_id', row.personnel_id 
                                            ,'first_name'  , row.first_name   
                                            ,'middle_name' , row.middle_name  
                                            ,'name'        , row.name         
                                            ,'birth'       , mask_date(row.birth)
                                            ,'country'     , row.country      
                                            ,'area_code'   , row.area_code    
                                            ,'phone'       , row.phone        
                                            ,'dept'        , row.dept         
                                            ,'job_title'   , row.job_title
                 ,get_periodic(isn=c1.cb.isn, gsn='AQ', esn='AR', lgn='curr_code', lsn = 3)
                 ,get_per_mult(isn=c1.cb.isn, gsn='AQ', esn='AT', lgn='bonus', lsn=9, upk=True, Int=True)                                
                                            ,"}")    

    lines+= 'Sequential Read by ISN returned '+str(count+1)+' record(s).'
except DataEnd:
    extraline+= 'Sequential Read by ISN returned '+str(count)+' record(s).'
    pass
except DatabaseError, (line, apa):
    extraline+='Database Error:'+line
                

In [49]:
line = lines.splitlines()

In [50]:
line[:5]

['{"isn": 283, "personnel_id": "11200308", "first_name": "MERIT", "middle_name": "","name": "PTAH", "birth": "17-10-1948", "country": "D","area_code": "06196", "phone": "299871", "dept": "FINA24","job_title": "SACHBEARBEITERIN", "curr_code": ["EUR", "EUR"], "bonus": [[1846, 2000, 500], [1743]]}',
 '{"isn": 284, "personnel_id": "11200309", "first_name": "TANIA", "middle_name": "ISAK","name": "DINESEN", "birth": "09-08-1944", "country": "D","area_code": "069", "phone": "988711", "dept": "FINA24","job_title": "BUCHHALTERIN", "curr_code": ["EUR"], "bonus": [[2564, 2000, 3800]]}',
 '{"isn": 285, "personnel_id": "11200312", "first_name": "JOHANNES", "middle_name": "MANFRED","name": "BLOEMER", "birth": "07-11-1979", "country": "D","area_code": "069", "phone": "775599", "dept": "FINA24","job_title": "SACHBEARBEITER", "curr_code": ["EUR", "EUR"], "bonus": [[2051, 1200, 2300], [1948]]}',
 '{"isn": 286, "personnel_id": "11200313", "first_name": "SIMON", "middle_name": "BENJAMIN","name": "LEIBACHE

In [51]:
parsed = json.loads(line[0].decode('utf-8'))

In [52]:
parsed

{u'area_code': u'06196',
 u'birth': u'17-10-1948',
 u'bonus': [[1846, 2000, 500], [1743]],
 u'country': u'D',
 u'curr_code': [u'EUR', u'EUR'],
 u'dept': u'FINA24',
 u'first_name': u'MERIT',
 u'isn': 283,
 u'job_title': u'SACHBEARBEITERIN',
 u'middle_name': u'',
 u'name': u'PTAH',
 u'personnel_id': u'11200308',
 u'phone': u'299871'}

In [36]:
parsed['bonus']

[[1846, 2000, 500], [1743]]